# Calculate yield statistics on simulation datasets

Statistical analysis consists of two stages of the topological gap protocol (TGP) described in Refs. [arXiv:2103.12217](https://arxiv.org/abs/2103.12217) and [arXiv:2207.02472](https://arxiv.org/abs/2207.02472):
* TGP Stage 1 identifies promising regions with stable zero bias peaks (ZBPs) and transfers these regions to TGP Stage 2;
* TGP Stage 2 searches clusters with stable ZBPs, topological gap, and surrounded by gapless boundary.

The simulation is performed for 0 mK, therefore we first temperature broaden the data. This temperature broadening is CPU intensive and will take ≈1 min per dataset. We save the broadened datasets in `simulated/yield/cached` such that it only has to be done once.

Note that we use 30 mK for broadening in Stage 1 and 40 mK in Stage 2. This is due to historical reasons. Increasing Stage 1 temperature to 40 mK barely changes regions for Stage 2.

In [ ]:
import os
THREAD_ENV_VARS = [
    "OMP_NUM_THREADS",
    "OPENBLAS_NUM_THREADS",
    "MKL_NUM_THREADS",
    "VECLIB_MAXIMUM_THREADS",
    "NUMEXPR_NUM_THREADS",
]
import xarray as xr

for k in THREAD_ENV_VARS:
    os.environ[k] = "1"
os.cpu_count()

In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
plt.rcParams.update(params)

In [ ]:
import numpy as np
from functools import partial
from pathlib import Path
from yield_analysis import analyze_1, analyze_2, parallel_map, show_roi2_tables, show_soi2_tables

import pandas as pd

In [ ]:
# if the notebook is in azure-quantum-tgp/notebooks, otherwise update the path as appropriate
folder = Path("../data/simulated/yield/")
folder_stage1 = folder / "stage1"
folder_stage2 = folder / "stage2"

fnames_stage1 = sorted(folder_stage1.glob("*.nc"))
print(f"{len(fnames_stage1)} .nc files total for stage 1")

fnames_stage2 = sorted(folder_stage2.glob("*.nc"))
print(f"{len(fnames_stage2)} .nc files total for stage 2")

if len(fnames_stage1) == 0 or fnames_stage1[0].stat().st_size < 1e6:
    raise Exception(f"You are missing the data, please see the Git LFS instructions at `{str(folder)}/README.md`")

# Analyze Stage 1 data

## Plot fraction of topo phase

In [ ]:
def get_SI_fraction(fname):
    ds = xr.load_dataset(fname)
    total_pts = ds.sizes["B"] * ds.sizes["V"]
    SI_pts = (((ds["L_SI"] < 0) | (ds["R_SI"] < 0)).sum("cutter_pair_index") > 0).sum().data
    return dict(**ds.attrs, fname=fname, total_pts=total_pts, SI_pts = SI_pts, ratio=SI_pts/total_pts)
    

In [ ]:
pd_pixels = pd.DataFrame(parallel_map(get_SI_fraction, fnames_stage1))

In [ ]:
# Plot the distribution of fraction of pixels with SI<0
fig = plt.figure(figsize=(6, 3))
cmap = mpl.colormaps['Set3']
bins = np.arange(0, 1+1e-9, 0.02)
groupby = ["sample_name", "surface_charge"]
colors = cmap(np.linspace(0, 1, len(pd_pixels.groupby(groupby))))
hist = np.zeros(len(bins)-1, dtype=float)
median_value = np.median(pd_pixels["ratio"])

plt.axvline(median_value, ls="--", lw=1, color="black")
for km, (k, gr) in enumerate(pd_pixels.groupby(groupby)):
    p_dict = {k : v for k, v in zip(groupby, k)}
    label = r"DLG-$\varepsilon$" if p_dict["sample_name"] == "simulated_DLG_epsilon" else r"SLG-$\beta$"
    hist1, _, _ = plt.hist(gr["ratio"], bins=bins, bottom=hist, color=colors[km], label=label + f"; {p_dict['surface_charge']}")
    hist += hist1

plt.text(median_value + 0.01, hist.max(), f"{median_value*100:.1f}%", ha="left", va="top")
plt.minorticks_on()
plt.xlim([0, 1])
plt.xlabel("Fraction of pixels with SI<0")
plt.ylabel("N samples")
plt.legend(fontsize=10, title="sample; disorder")
plt.tight_layout()
fig.savefig("topo_pixel_dist.jpg", dpi=200)

## TGP Stage 1
Now we will perform the analysis on all of these datasets.
*(Stage 1 can take 3-10 minutes on a recent laptop.)*

In [ ]:
T_mK = 30
f = partial(analyze_1, T_mK=T_mK)
results_1 = parallel_map(f, fnames_stage1)
df_1 = pd.DataFrame(results_1)
df_1_passed = df_1[~df_1.V_min.isna()]

In [ ]:
# Print all the bounding boxes
cols = [
    "sample_name",
    "disorder_seed",
    "geometry_seed",
    "surface_charge",
    "B_min",
    "V_min",
    "B_max",
    "V_max",
]
df_1_passed[cols]

In [ ]:
# Print the number of passed devices
passed_devices = df_1_passed.groupby(["sample_name", "surface_charge"]).apply(len)
total = df_1.groupby(["sample_name", "surface_charge"]).apply(len)
df = pd.concat([passed_devices, total], axis=1)
df.columns = ["# passed", "total"]
df

# TGP Stage 2
*(Stage 2 can take 2-4 hours on a recent laptop.)*

Change `B_max` to `2.5` to get the statistics for the "Yield <2.5T" column.

In [ ]:
T_mK = 40
f = partial(analyze_2, T_mK=T_mK, return_datasets=False)
all_results = parallel_map(f, fnames_stage2)
results = [r for r in all_results if r is not None]

In [ ]:
df_stats = pd.DataFrame(results)
tables = show_roi2_tables(df_stats)

## Plot B range distribution

In [ ]:
def get_B_range(fname):
    ds = xr.load_dataset(fname)
    return dict(**ds.attrs, fname=fname,  Bmax = ds.B.max().data, Bmin= ds.B.min().data, dB = ds.B.max().data - ds.B.min().data)

_data_B_range = parallel_map(get_B_range, fnames_stage2)
df_dB = pd.DataFrame(_data_B_range) 
ds_dB = df_dB[["sample_name", "Bmin", "Bmax", "dB"]].set_index("sample_name").to_xarray()

In [ ]:
# Load exp data
stacks = {
    "A1" : r"SLG-$\beta$",
    "A2" : r"SLG-$\beta$",
    "A3" : r"SLG-$\beta$",
    "B" : r"DLG-$\varepsilon$",
    "C" : r"DLG-$\delta$",
    "D" : r"DLG-$\delta$",
    "E" : r"DLG-$\delta$"
}
_data_range_exp = []
for label in ["A1", "A2", "A3", "B", "C", "D", "E"]:
    name = f"device{label}_stage2"
    ds_exp = xr.load_dataset(folder / ".." / ".." / "experimental" / f"{name}_left.nc")
    dB = ds_exp["B"].max().data - ds_exp["B"].min().data
    
    _data_range_exp += [dict(
        label=label, 
        stack=stacks[label],        
        dB=dB, 
        
    )]
df_exp = pd.DataFrame(_data_range_exp).set_index("label")

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

fig = plt.figure(figsize=(6, 3))
ax = fig.add_subplot(111)

bins = np.arange(0, 2.8, 0.15)
SLG_beta_hist, _, _r = plt.hist(ds_dB["dB"].sel(sample_name="simulated_SLG_beta"), bins=bins, color="lightblue", label=r"SLG-$\beta$")
DLG_eps_hist, _, _r = plt.hist(ds_dB["dB"].sel(sample_name="simulated_DLG_epsilon"), bins=bins, color="lightgreen", label=r"DLG-$\varepsilon$", bottom=SLG_beta_hist)
hist, _, _ = plt.hist(ds_dB["dB"], bins=bins, color="black", label=None, histtype="step")


plt.plot([df_exp.loc["A1"]["dB"], df_exp.loc["A1"]["dB"]], [0, np.max(hist)*0.4], ls="--", color="dodgerblue")
ax.text(df_exp.loc["A1"]["dB"], np.max(hist)*0.4, "A1", ha="center", va="bottom", color="dodgerblue")
plt.plot([df_exp.loc["A2"]["dB"], df_exp.loc["A2"]["dB"]], [0, np.max(hist)*0.4], ls="--", color="dodgerblue")
ax.text(df_exp.loc["A2"]["dB"], np.max(hist)*0.4, "A2", ha="center", va="bottom", color="dodgerblue")
plt.plot([df_exp.loc["A3"]["dB"], df_exp.loc["A3"]["dB"]], [0, np.max(hist)*0.4], ls="--", color="dodgerblue")
ax.text(df_exp.loc["A3"]["dB"], np.max(hist)*0.4, "A3", ha="center", va="bottom", color="dodgerblue")

plt.plot([df_exp.loc["B"]["dB"], df_exp.loc["B"]["dB"]], [0, np.max(hist)*0.35], ls="-.", color="green")
ax.text(df_exp.loc["B"]["dB"], np.max(hist)*0.35, "B", ha="center", va="bottom", color="green")

plt.plot([df_exp.loc["C"]["dB"], df_exp.loc["C"]["dB"]], [0, np.max(hist)*0.3], ls="-.", color="darkgreen")
ax.text(df_exp.loc["C"]["dB"], np.max(hist)*0.3, "C", ha="center", va="bottom", color="darkgreen")
plt.plot([df_exp.loc["D"]["dB"], df_exp.loc["D"]["dB"]], [0, np.max(hist)*0.3], ls="-.", color="darkgreen")
ax.text(df_exp.loc["D"]["dB"], np.max(hist)*0.3, "D", ha="center", va="bottom", color="darkgreen")
plt.plot([df_exp.loc["E"]["dB"], df_exp.loc["E"]["dB"]], [0, np.max(hist)*0.3], ls="-.", color="darkgreen")
ax.text(df_exp.loc["E"]["dB"], np.max(hist)*0.3, "E", ha="center", va="bottom", color="darkgreen")

#ax.bar(bins[:-1], SLG_beta_hist, width=np.diff(bins), edgecolor=None, align="edge", color="lightblue")

plt.legend()
plt.xlabel("$\Delta B$ [T] (simulations)")
plt.ylabel("N samples")
plt.minorticks_on()

plt.tight_layout()
plt.show()
fig.savefig("range_distribution.jpg", dpi=200)

## Threshold stability
We vary the gap threshold factor by $\pm 0.01$ and check the FDR

In [ ]:
T_mK = 40
f = partial(analyze_2, T_mK=T_mK, return_datasets=False, gap_threshold_factor=0.04)
all_results = parallel_map(f, fnames_stage2)
results = [r for r in all_results if r is not None]

In [ ]:
df_stats = pd.DataFrame(results)
show_roi2_tables(df_stats)

In [ ]:
T_mK = 40
f = partial(analyze_2, T_mK=T_mK, return_datasets=False, gap_threshold_factor=0.06)
all_results = parallel_map(f, fnames_stage2)
results = [r for r in all_results if r is not None]

In [ ]:
df_stats = pd.DataFrame(results)
show_roi2_tables(df_stats)

## FDR with Bmax

In [ ]:
T_mK = 40
f = partial(analyze_2, T_mK=T_mK, return_datasets=False, B_max=2.5)
all_results = parallel_map(f, fnames_stage2)
results = [r for r in all_results if r is not None]

In [ ]:
df_stats = pd.DataFrame(results)
show_roi2_tables(df_stats);

## Individual cluster check

In [ ]:
import tgp
import toolz
import numpy as np
T_mK=40
f = partial(analyze_2, T_mK=T_mK, return_datasets=True)

Bmax_arr = np.arange(2.5, 3.01, 0.05)
B_ref = 3.0
B_roi_max = 2.0

keys = ["sample_name", "surface_charge", "geometry_seed", "disorder_seed"]

def check_stability(fname):
    # We take analysis cropped at B_max = B_ref and find all clusters that extend up to B_roi_max. 
    # This is the reference dataset. 
    # Then we reanalyze the data with different cropping boxes check if there's an overlap 
    # for every cluster that we identified in the reference dataset.
    
    ret = f(fname, B_max = B_ref)
    if ret is None:
        return None
        
    ds = ret["zbp_ds"]

    if ds.roi2.max() == 0:
        return None

    roi2_ref = ds.roi2
    # Find boolean mask of every cluster that is entirely at B<=B_roi_max, use that as a reference dataset.
    # We will only track these clusters.
    masks = []
    for k in range(1, int(roi2_ref.max().data)+1):
        if roi2_ref.where((roi2_ref == k), drop=True).B.max() <= B_roi_max:
           masks += [(roi2_ref == k) & (roi2_ref.B <= B_roi_max)]
    if len(masks) == 0:
        masks = [roi2_ref * 0]

    attrs = xr.load_dataset(fname).attrs
    _additional = toolz.keyfilter(lambda x: x in keys, attrs)

    data = []

    for B_max in Bmax_arr:
        if np.abs(B_max - B_ref) > 1e-9:
            ret_comp = f(fname, B_max = B_max)
        else:
            ret_comp = ret

        if ret_comp is not None:
            roi2_comp = ret_comp["zbp_ds"].roi2
        else:
            roi2_comp = ds.roi2 * 0

        # Find boolean masks of all clusters, crop to B <= B_roi_max. We allow clusters to extend past B_roi_max
        masks_comp = [(roi2_comp == k) & (roi2_comp.B <= B_roi_max) for k in range(1, roi2_comp.max().data + 1)]
        if len(masks_comp) == 0:
            masks_comp = [roi2_comp * 0]

        overlap_cl = 0
        for mask in masks:
            overlapping_rois = roi2_comp.where(mask, drop=True).data.reshape(-1)
            overlapping_rois = overlapping_rois[~np.isnan(overlapping_rois)] != 0
            overlap_cl += (np.sum(overlapping_rois) / len(overlapping_rois)) > 0.0
            
        data += [dict(
            **_additional, 
            Bmax=B_max, 
            ref_count = len(masks),
            overlap_cl=overlap_cl,
        )
        ]

    return pd.DataFrame(data)

In [ ]:
fnames_SLG_beta = sorted(list(filter(lambda x: "SLG" in x.name, fnames_stage2)))
len(fnames_SLG_beta)

In [ ]:
stability_all_results = parallel_map(check_stability, fnames_SLG_beta, max_workers=32)
df_stab = pd.concat([r for r in stability_all_results if r is not None])

In [ ]:
ds1 = df_stab[["sample_name", "surface_charge", "Bmax", "ref_count", "overlap_cl"]].groupby(["sample_name", "surface_charge", "Bmax"]).sum().to_xarray()
# Plot stability of ROI2s
fig = plt.figure(figsize=(6, 3))
ax1 = fig.add_subplot(121)
ax1.text(0.03, 0.03, 'a', ha='left', verticalalignment='bottom', transform=ax1.transAxes, weight="bold", fontsize=16)
plt.minorticks_on()
ax2 = fig.add_subplot(122, sharex=ax1)
ax2.text(0.03, 0.03, 'b', ha='left', verticalalignment='bottom', transform=ax2.transAxes, weight="bold", fontsize=16)

plt.minorticks_on()
markers = ["o", "*", "p"]
for n, (sc, _ds) in enumerate(ds1["overlap_cl"].groupby("surface_charge")):
    ax1.plot(_ds["Bmax"], _ds.data.reshape(-1), marker=markers[n], ms=8)
ax1.set_title("")
ax1.set_ylabel("#ROI2")
ax1.set_xlabel(r"$B_{max}$ [T]")
ax1.set_ylim([0, None])

ax1.grid()
plt.rc('legend',fontsize=10)
for n, (sc, _ds) in enumerate(ds1["overlap_cl"].groupby("surface_charge")):
    ax2.plot(_ds["Bmax"], (_ds / _ds.sel(Bmax=B_ref, method="nearest")).data.reshape(-1), marker=markers[n], ms=8, label=f"{sc}")

ax2.legend(fontsize=12, title="surface charge", loc=4)
ax2.set_title("")
ax2.set_ylabel("Fraction of ROI2", fontsize=14)
ax2.set_xlabel(r"$B_{max}$ [T]")
ax2.set_ylim([0.0, 1.0])
ax2.grid()

plt.tight_layout()
plt.show()
fig.savefig("field_range_sensitivity.jpg", dpi=200)

In [ ]:
import gc
gc.collect()